# Turi Create

https://www.raywenderlich.com/5653-create-ml-tutorial-getting-started    
https://github.com/apple/turicreate        
https://apple.github.io/turicreate/docs/userguide/    

#### Turi Create Image Classifier    
https://apple.github.io/turicreate/docs/userguide/image_classifier/

#### Text Classifier    
https://apple.github.io/turicreate/docs/userguide/text_classifier/
https://www.raywenderlich.com/5213-natural-language-processing-on-ios-with-turi-create

# Turi Create Image Similarity

Turi Create에는 5가지의 task-focused toolkit이 있으며 아직 Create ML은 포함되어 있지 않다.
- Recommender systems
- Image similarity
- Object detection
- Style transfer
- Activity classification

여기서 테스트할 개와 고양이 분류는 similar image 에 해당한다.

In [3]:
import turicreate as tc #turicreate을 불러온다.

reference_data = tc.image_analysis.load_images('./PetImages') #이미지를 table에 로드
#https://www.microsoft.com/en-us/download/details.aspx?id=54765 이 데이터를 PetImages 폴더에 저장한 후 실행
reference_data = reference_data.add_row_number() #row number를 추가한다.
reference_data.save('./kaggle-pets.sframe') #데이터 저장

Unsupported image format. Supported formats are JPEG and PNG	 file: /Users/geunseong-gai/Desktop/PetImages/.DS_Store

Unexpected JPEG decode failure	 file: /Users/geunseong-gai/Desktop/PetImages/Cat/140.jpg

Unexpected JPEG decode failure	 file: /Users/geunseong-gai/Desktop/PetImages/Cat/23.jpg

Unexpected JPEG decode failure	 file: /Users/geunseong-gai/Desktop/PetImages/Cat/391.jpg

Unexpected JPEG decode failure	 file: /Users/geunseong-gai/Desktop/PetImages/Cat/445.jpg

Unexpected JPEG decode failure	 file: /Users/geunseong-gai/Desktop/PetImages/Dog/296.jpg

Unexpected JPEG decode failure	 file: /Users/geunseong-gai/Desktop/PetImages/Dog/414.jpg

Unexpected JPEG decode failure	 file: /Users/geunseong-gai/Desktop/PetImages/Dog/50.jpg

In [4]:
reference_data.explore() #id, path, image 를 가지고 있는 table window를 확인할 수 있다.

Materializing SFrame...

Done.

In [5]:
model = tc.image_similarity.create(reference_data) #reference_data(table)로 모델을 생성한다.

Download completed: /var/tmp/model_cache/resnet-50-symbol.json
Download completed: /var/tmp/model_cache/resnet-50-0000.params
Resizing images...
Performing feature extraction on resized images...
Completed 512/993
Completed 993/993


<unknown>:39: DeprecationWarning: invalid escape sequence \s


Starting brute force nearest neighbors model training.

# Unsupervised Learning

레이블이 분류된 데이터를 사용해 image classifier 하는 것을 supervised learning이라 한다.    
여기에서는 레이블이 분류된 데이터를 사용하지만, 레이블을 사용하지 않고 unsupervised learning을 구현한다.    
기본 모델은 이미 많은 이미지의 유사한 feature를 클러스터링할 수 있도록 학습되어 있다. 이후 nearest neighbors model 훈련을 한다.    
각 이미지에 대해 다른 모든 이미지와의 거리를 계산하고 다른 이미지를 radius band로 묶는다.

# Querying the Model

In [6]:
query_results = model.query(reference_data[0:10], k=10)
#reference_data의 첫 10행이 포함된 배열을 전달한다. 그리고 각 10개씩 similar image를 쿼리한다.
query_results.head() #첫 10개의 행을 표시한다.

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 10      | 0.100705    | 30.767ms     |

| Done         |         | 100         | 89.337ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,0,0.0,1
0,309,14.810596930958411,2
0,30,14.816778324847382,3
0,49,14.991121129639563,4
0,485,15.443226328039263,5
0,344,15.4770148233204,6
0,330,15.596135974358623,7
0,331,15.608336957399032,8
0,76,15.625666884523177,9
0,105,15.756726899354806,10


In [7]:
reference_data[9]['image'].show() #10번째 이미지(label이 9)와 비슷한 이미지를 확인한다.

In [10]:
similar_rows = query_results[query_results['query_label'] == 9]['reference_label'] 
#실제로 반환된 10번째 이미지(label이 9) 비슷한 이미지들의 reference_label(id)을 가져온다.
print(similar_rows)
reference_data.filter_by(similar_rows, 'id').explore() #해당 reference_label(id)를 필터링해 보여준다.

[9, 336, 442, 334, 426, 276, 289, 15, 301, 28, ... ]


Materializing SFrame...

Done.